#### Note

In [ ]:
# pd.set_option('display.max_rows', 100) # 최대 display option set
# pd.reset_option('display.max_rows') # 최대 display option reset

# from datetime import timedelta
# datetime.datetime(2019,12,31) - datetime.datetime(2008,1,1)

#### IMPORT

In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### Air data

##### index 생성

In [3]:
# r 코드 돌려서 만든 air_2020_idx 가져오기

air_2020_idx_zio = pd.read_csv('air\\air_2020_idx_zio.csv', encoding = 'euc-kr')
air_2020_idx_zio

,주소,측정소코드,air_out_idx
0,강원 강릉시 경강로 2179(옥천동 327-2),632132,4
1,강원 고성군 간성읍,632421,17
2,강원 고성군 간성읍 수성로 160,632441,17
3,강원 동해시 천곡로 77(천곡동 806),632151,77
4,강원 삼척시 남양길 11(남양동 331-9),632161,102
...,...,...,...
657,충청북도 청주시 상당구 가덕면 보청대로 4650,633215,221
658,충청북도 청주시 서원구 원흥로 81,633213,222
659,충청북도 청주시 흥덕구 오송읍 오송생명로 150,633214,224
660,충청북도 충주시 살미면 세성양지말길 41,633125,226


In [4]:
# qgis 지도 idx - 72개 dbf 파일로 수정

from dbfread import DBF

idx = DBF('spatial\\Z_SOP_BND_SIGUNGU_sudogwon.dbf', encoding = 'euc-kr')
idx = pd.DataFrame(idx)
idx['OBJECTID'] = idx['OBJECTID'].astype(int)
idx
# idx = pd.read_csv('spatial\\qgis_지도_idx_75.csv', encoding = 'euc-kr')
# idx

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,OBJECTID,check
0,20200630,31370,가평군,1,1
1,20200630,11230,강남구,2,1
2,20200630,11250,강동구,3,1
3,20200630,11090,강북구,5,1
4,20200630,11160,강서구,6,1
...,...,...,...,...,...
67,20200630,31200,파주시,231,1
68,20200630,31070,평택시,233,1
69,20200630,31270,포천시,234,1
70,20200630,31180,하남시,237,1


In [5]:
# merge air idx with qgis_지도_idx

map_air_idx = pd.merge(idx, air_2020_idx_zio[['air_out_idx', '측정소코드']], left_on='OBJECTID', right_on='air_out_idx', how = 'left')
map_air_idx

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,OBJECTID,check,air_out_idx,측정소코드
0,20200630,31370,가평군,1,1,1,131611
1,20200630,31370,가평군,1,1,1,131612
2,20200630,11230,강남구,2,1,2,111263
3,20200630,11230,강남구,2,1,2,111261
4,20200630,11250,강동구,3,1,3,111274
...,...,...,...,...,...,...,...
188,20200630,31240,화성시,247,1,247,131555
189,20200630,31240,화성시,247,1,247,131557
190,20200630,31240,화성시,247,1,247,131558
191,20200630,31240,화성시,247,1,247,131556


In [6]:
map_air_idx.drop_duplicates(subset=['측정소코드'], inplace = True)   # 여기서 air_out_idx 로 drop해버리면 안됨
map_air_idx.reset_index(drop = True, inplace = True)
map_air_idx

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,OBJECTID,check,air_out_idx,측정소코드
0,20200630,31370,가평군,1,1,1,131611
1,20200630,31370,가평군,1,1,1,131612
2,20200630,11230,강남구,2,1,2,111263
3,20200630,11230,강남구,2,1,2,111261
4,20200630,11250,강동구,3,1,3,111274
...,...,...,...,...,...,...,...
177,20200630,31240,화성시,247,1,247,131554
178,20200630,31240,화성시,247,1,247,131552
179,20200630,31240,화성시,247,1,247,131557
180,20200630,31240,화성시,247,1,247,131558


In [7]:
# 겹치는 column drop

map_air_idx.drop(columns = {'SIGUNGU_CD', 'SIGUNGU_NM', 'OBJECTID', 'check'}, inplace = True)
map_air_idx

,BASE_DATE,air_out_idx,측정소코드
0,20200630,1,131611
1,20200630,1,131612
2,20200630,2,111263
3,20200630,2,111261
4,20200630,3,111274
...,...,...,...
177,20200630,247,131554
178,20200630,247,131552
179,20200630,247,131557
180,20200630,247,131558


##### air data 불러오기

In [13]:
# air data 불러와서 합치기 - 9분

year_list = np.arange(2012, 2018, 1).tolist()
year_list = list(map(str, year_list))
year_list

air_data = pd.DataFrame()
for year in year_list: 
    air_tmp = pd.read_excel('D:\\SNUlab\\0. data\\out of hospital sudden cardiac arrest\\air_year_{}.xlsx'.format(year))
    air_data = pd.concat([air_data, air_tmp], axis = 0)

In [23]:
# 12년은 11월, 12월만 뽑기

air_data = air_data.loc[air_data['dt'] >= '2012-11-01'].reset_index(drop = True)

In [24]:
# air_data.to_csv('air\\air_1217.csv', encoding = 'euc-kr', index = False)

##### air_data 다시 불러오기 (2012.11 ~ 2017.12)

In [8]:
# air_data 다시 불러오기 

air_data = pd.read_csv('air\\air_1217.csv', encoding = 'euc-kr')

In [9]:
# '주소' 겹치니까 지우기

air_data.drop(columns = ['주소'], axis = 1, inplace = True)

In [ ]:
air_data.isna().sum()

In [14]:
air_data_test = air_data.loc[air_data['PM25_mean'].isna() == False].reset_index(drop = True)
len(air_data_test['측정소코드'].unique())

264

In [15]:
air_data_test = air_data.loc[air_data['PM10_mean'].isna() == False].reset_index(drop = True)
len(air_data_test['측정소코드'].unique())

347

In [11]:
len(air_data['측정소코드'].unique())

356

In [224]:
# datetime 으로 변경

air_data['dt'] = pd.to_datetime(air_data['dt'])

In [225]:
# describe

air_data.describe()

,측정소코드,SO2_count,SO2_mean,SO2_median,SO2_min,SO2_max,SO2_sum,CO_count,CO_mean,CO_median,...,PM10_median,PM10_min,PM10_max,PM10_sum,PM25_count,PM25_mean,PM25_median,PM25_min,PM25_max,PM25_sum
count,603843.000000,603843.000000,585341.000000,585341.000000,585341.000000,585341.000000,603843.000000,603843.000000,584111.000000,584111.000000,...,581950.000000,581950.000000,581950.000000,603843.000000,469683.000000,184846.000000,184846.000000,184846.000000,184846.000000,469683.000000
mean,334848.026727,22.849241,0.004866,0.004494,0.003157,0.008700,0.111191,22.768388,0.514528,0.495662,...,46.320333,26.746348,76.106848,1079.258267,9.113289,25.407879,24.591314,13.088631,41.891358,231.998084
std,230357.906593,4.434138,0.002969,0.002644,0.001649,0.009254,0.072124,4.583666,0.237967,0.234486,...,26.937303,18.245045,44.740448,663.769376,11.418916,14.115665,14.394357,10.106344,22.032741,356.463178
min,111121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,131241.000000,24.000000,0.003000,0.003000,0.002000,0.004000,0.068000,24.000000,0.354200,0.300000,...,29.000000,14.000000,48.000000,670.000000,0.000000,15.416700,14.500000,6.000000,26.000000,0.000000
50%,238146.000000,24.000000,0.004300,0.004000,0.003000,0.007000,0.100000,24.000000,0.470800,0.450000,...,41.000000,23.000000,67.000000,972.000000,0.000000,22.791700,22.000000,11.000000,38.000000,0.000000
75%,525112.000000,24.000000,0.006000,0.006000,0.004000,0.010000,0.140000,24.000000,0.625000,0.600000,...,57.500000,35.000000,93.000000,1371.000000,24.000000,32.375000,31.500000,18.000000,53.000000,433.000000
max,831492.000000,24.000000,0.282600,0.330000,0.047000,0.502000,6.218000,24.000000,6.100000,6.100000,...,867.000000,328.000000,1484.000000,19083.000000,24.000000,144.458300,146.500000,122.000000,843.000000,3467.000000


In [233]:
# map_air_idx + air_data

air_data_idx = pd.merge(air_data, map_air_idx[['측정소코드', 'air_out_idx']], on="측정소코드", how = 'left')

In [234]:
len(map_air_idx['air_out_idx'].unique())

72

In [235]:
len(air_data_idx['air_out_idx'].unique())

73

In [227]:
air_data_idx.loc[air_data_idx['air_out_idx'] == 114, '측정소코드'].unique()

array([131120, 131123, 131124, 131128], dtype=int64)

In [204]:
# nan 은 비수도권

air_data_idx['air_out_idx'].unique()

array([ nan,   1.,  19.,  20.,  26.,  28.,  31.,  36.,  40.,  47.,  55.,
        74.,  92., 114., 115., 116., 128., 126., 127., 125., 131., 135.,
       136., 137., 139., 138., 143., 150., 163., 171., 170., 169., 181.,
       182., 183., 231., 233., 234., 237., 247., 197., 168., 196.,  32.,
       117., 203.,  73., 118.,  66., 177., 110.,  78.,   6.,  35., 155.,
        76.,  27.,   2., 113., 122.,   3.,  43.,   5., 144.,  58.,  15.,
        83.,  54.,  69.,  93., 106., 148., 200.])

In [205]:
# 고양시 일산서구 (21) 만 없음

a = set(map_air_idx['air_out_idx'].unique())
b = set(air_data_idx['air_out_idx'].unique())
a - b

set()

In [239]:
# 일자별, date 별 mean (구별 측정소가 여러개)

air_data_idx = pd.DataFrame(air_data_idx.groupby(['air_out_idx', 'dt']).mean()).reset_index()
air_data_idx

,air_out_idx,dt,측정소코드,SO2_count,SO2_mean,SO2_median,SO2_min,SO2_max,SO2_sum,CO_count,...,PM10_median,PM10_min,PM10_max,PM10_sum,PM25_count,PM25_mean,PM25_median,PM25_min,PM25_max,PM25_sum
0,1.0,2012-11-01,131611.0,24.000000,0.004300,0.004000,0.003000,0.007000,0.102000,24.000000,...,49.500000,25.000000,63.000000,1080.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2012-11-02,131611.0,24.000000,0.002800,0.003000,0.002000,0.004000,0.067000,24.000000,...,32.000000,22.000000,54.000000,825.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2012-11-03,131611.0,24.000000,0.002800,0.003000,0.002000,0.004000,0.067000,24.000000,...,31.500000,23.000000,81.000000,914.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2012-11-04,131611.0,24.000000,0.003300,0.003000,0.002000,0.004000,0.080000,24.000000,...,35.000000,31.000000,61.000000,942.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2012-11-05,131611.0,24.000000,0.002800,0.003000,0.002000,0.003000,0.068000,24.000000,...,26.500000,17.000000,50.000000,718.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135859,247.0,2017-12-27,131552.0,16.000000,0.005850,0.005500,0.004500,0.010000,0.093333,24.000000,...,31.000000,20.000000,46.666667,757.000000,24.000000,16.180567,16.166667,7.333333,28.000000,388.333333
135860,247.0,2017-12-28,131552.0,16.000000,0.007450,0.007250,0.004500,0.011500,0.119000,24.000000,...,62.000000,40.666667,97.666667,1567.666667,24.000000,39.819467,35.000000,21.666667,71.666667,955.666667
135861,247.0,2017-12-29,131552.0,17.666667,0.006633,0.006667,0.004667,0.009000,0.126333,23.666667,...,98.666667,77.333333,130.666667,2416.333333,22.333333,74.570433,70.500000,57.333333,97.000000,1669.666667
135862,247.0,2017-12-30,131552.0,23.000000,0.005733,0.006000,0.004667,0.008000,0.130667,24.000000,...,126.500000,103.666667,145.333333,2590.000000,24.000000,100.444467,101.500000,84.666667,119.000000,2410.666667


In [240]:
air_data_idx.describe()

,air_out_idx,측정소코드,SO2_count,SO2_mean,SO2_median,SO2_min,SO2_max,SO2_sum,CO_count,CO_mean,...,PM10_median,PM10_min,PM10_max,PM10_sum,PM25_count,PM25_mean,PM25_median,PM25_min,PM25_max,PM25_sum
count,135864.000000,135864.000000,135864.000000,135342.000000,135342.000000,135342.000000,135342.000000,135864.000000,135864.000000,135306.000000,...,134305.000000,134305.000000,134305.000000,135864.000000,105192.000000,54853.000000,54853.000000,54853.000000,54853.000000,105192.000000
mean,110.027778,210988.674288,23.297517,0.005245,0.005001,0.003683,0.008272,0.122200,23.199324,0.566627,...,49.292297,28.803097,80.336234,1161.817225,8.614935,25.794798,24.890839,13.271559,42.639284,222.355368
std,66.667115,232069.252412,2.550924,0.002105,0.001979,0.001298,0.005751,0.050978,2.669131,0.220377,...,28.776265,19.016345,47.986054,693.130305,9.421708,14.362798,14.582223,10.335018,22.366776,304.198570
min,1.000000,111122.500000,0.000000,0.000600,0.000600,0.000000,0.000700,0.000000,0.000000,0.100000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.250000,111262.750000,24.000000,0.003800,0.003667,0.003000,0.005000,0.088000,24.000000,0.416700,...,31.500000,16.000000,52.000000,736.187500,0.000000,15.869600,15.000000,6.000000,27.000000,0.000000
50%,116.500000,131178.000000,24.000000,0.004900,0.004750,0.003500,0.007000,0.114333,24.000000,0.525000,...,44.000000,25.000000,71.000000,1044.000000,7.000000,23.166700,22.000000,11.000000,38.000000,82.200000
75%,157.000000,131479.375000,24.000000,0.006250,0.006000,0.004500,0.009667,0.147000,24.000000,0.666700,...,60.750000,37.500000,97.000000,1449.500000,14.000000,32.791700,32.000000,18.000000,54.000000,365.000000
max,247.000000,831153.000000,24.000000,0.032533,0.026000,0.017000,0.252667,0.760333,24.000000,2.608350,...,867.000000,328.000000,1238.000000,19083.000000,24.000000,135.083300,142.000000,104.000000,225.000000,3242.000000


In [241]:
# idx_list = air_data_idx['air_out_idx'].unique().tolist()

# for ix in idx_list: 
#     tmp = air_data_idx.loc[air_data_idx['air_out_idx'] == ix]
#     print('idx: ', ix, '------------------------------')
#     for col in tmp.columns: 
#         print(col, tmp[col].isna().sum())
#     print('-------------------------')

In [242]:
air_data_idx.loc[air_data_idx['air_out_idx'] == 114, 'O3_mean'].describe()

count    1887.000000
mean        0.020472
std         0.011692
min         0.001900
25%         0.010850
50%         0.018367
75%         0.028400
max         0.074433
Name: O3_mean, dtype: float64

In [243]:
air_data_idx.loc[air_data_idx['air_out_idx'] == 115, 'O3_mean'].describe()

count    1887.000000
mean        0.020199
std         0.011296
min         0.002100
25%         0.011000
50%         0.018700
75%         0.027600
max         0.073633
Name: O3_mean, dtype: float64

In [244]:
air_data_idx.columns

Index(['air_out_idx', 'dt', '측정소코드', 'SO2_count', 'SO2_mean', 'SO2_median',
       'SO2_min', 'SO2_max', 'SO2_sum', 'CO_count', 'CO_mean', 'CO_median',
       'CO_min', 'CO_max', 'CO_sum', 'O3_count', 'O3_mean', 'O3_median',
       'O3_min', 'O3_max', 'O3_sum', 'NO2_count', 'NO2_mean', 'NO2_median',
       'NO2_min', 'NO2_max', 'NO2_sum', 'PM10_count', 'PM10_mean',
       'PM10_median', 'PM10_min', 'PM10_max', 'PM10_sum', 'PM25_count',
       'PM25_mean', 'PM25_median', 'PM25_min', 'PM25_max', 'PM25_sum'],
      dtype='object')

In [245]:
# 성남시 분당구 (114) O3 결측 -> 성남시 중원구 (115) O3 로 채우기
# 수정구로 변경 (115)

bundang = air_data_idx.loc[air_data_idx['air_out_idx'] == 114].reset_index(drop = True)
bundang_O3 = bundang[['dt', 'O3_count', 'O3_mean', 'O3_median', 'O3_min', 'O3_max', 'O3_sum']]
bundang.drop(['O3_count', 'O3_mean', 'O3_median', 'O3_min', 'O3_max', 'O3_sum'], axis = 1, inplace = True)

sujeong = air_data_idx.loc[air_data_idx['air_out_idx'] == 115].reset_index(drop = True)
sujeong_O3 = sujeong[['dt', 'O3_count', 'O3_mean', 'O3_median', 'O3_min', 'O3_max', 'O3_sum']]

bundang = pd.merge(bundang, sujeong_O3, on='dt', how = 'left')
bundang.reset_index(drop = True, inplace = True)

air_data_idx = air_data_idx.loc[air_data_idx['air_out_idx'] != 114].reset_index(drop = True)
air_data_idx = pd.concat([air_data_idx, bundang], axis = 0)
air_data_idx.reset_index(drop = True, inplace = True)

air_data_idx.loc[air_data_idx['air_out_idx'] == 114, 'O3_mean'].describe()

count    1887.000000
mean        0.020199
std         0.011296
min         0.002100
25%         0.011000
50%         0.018700
75%         0.027600
max         0.073633
Name: O3_mean, dtype: float64

In [246]:
air_data_idx.loc[air_data_idx['air_out_idx'] == 114].isna().sum()

air_out_idx      0
dt               0
측정소코드            0
SO2_count        0
SO2_mean         0
SO2_median       0
SO2_min          0
SO2_max          0
SO2_sum          0
CO_count         0
CO_mean          0
CO_median        0
CO_min           0
CO_max           0
CO_sum           0
O3_count         0
O3_mean          0
O3_median        0
O3_min           0
O3_max           0
O3_sum           0
NO2_count        0
NO2_mean         0
NO2_median       0
NO2_min          0
NO2_max          0
NO2_sum          0
PM10_count       0
PM10_mean        0
PM10_median      0
PM10_min         0
PM10_max         0
PM10_sum         0
PM25_count     426
PM25_mean      791
PM25_median    791
PM25_min       791
PM25_max       791
PM25_sum       426
dtype: int64

In [ ]:
air_data_idx  # len same check

In [18]:
# # idx 21 추가 - 취소, 21이 없어야 함
# # 고양시 일산동구(20) 복사해서 고양시 일산서구(21) 로 한번 더 붙이기

# air_20 = air_data_idx.loc[air_data_idx['air_out_idx'] == 20].reset_index(drop = True)
# air_20['air_out_idx'] = 21

# air_data_idx = pd.concat([air_data_idx, air_20], axis = 0).reset_index(drop = True)

#### Weather data

##### weather index 불러오기
- 수도권 73개만 있음 (air_out_idx)      
- 01.14 양평, 여주 추가되어 71 -> 73

In [29]:
weather_index = pd.read_csv('weather\\weather_index.csv', encoding = 'euc-kr')
weather_index

,지점,지점명,air_out_idx
0,505,가평조종,1
1,505,현리,1
2,505,가평조종,1
3,1524,설악활곡,1
4,1428,명지산주차장,1
...,...,...,...
849,547,양평군,145
850,326,양평군,145
851,573,양평군,145
852,574,여주시,147


In [30]:
# 21, 69번 없는 거 확인

len(weather_index['air_out_idx'].unique())

73

-------------

75개로 다시 만든 qgis 지도 idx 불러와서 안 겹치는 거 뭔지 빼보자 - 확인 완료 (안해도 됨)

In [5]:
qgis_지도_idx_75 = pd.read_csv('spatial\\qgis_지도_idx_75.csv', encoding = 'euc-kr')
qgis_지도_idx_75

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,OBJECTID,check
0,20200630,31104,고양시 일산서구,21,1
1,20200630,31380,양평군,145,1
2,20200630,31280,여주시,147,1
3,20200630,31370,가평군,1,1
4,20200630,11230,강남구,2,1
...,...,...,...,...,...
70,20200630,31200,파주시,231,1
71,20200630,31070,평택시,233,1
72,20200630,31270,포천시,234,1
73,20200630,31180,하남시,237,1


In [6]:
a = set(qgis_지도_idx_75['OBJECTID'].unique())
b = set(weather_index['air_out_idx'].unique())
a-b

{21, 69}

In [7]:
nonmatched = list(a - b)
nonmatched
qgis_지도_idx_75.loc[qgis_지도_idx_75['OBJECTID'].isin(nonmatched)]

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,OBJECTID,check
0,20200630,31104,고양시 일산서구,21,1
25,20200630,23020,동구,69,1


----------------------

##### weather_index 전처리

In [31]:
weather_index.drop_duplicates(subset=['지점'], inplace = True)   # 지점 기준 drop dup - 627개
weather_index.reset_index(drop = True, inplace = True)
weather_index

,지점,지점명,air_out_idx
0,505,가평조종,1
1,1524,설악활곡,1
2,1428,명지산주차장,1
3,1432,호명호수,1
4,1256,청평,1
...,...,...,...
622,547,양평군,145
623,326,양평군,145
624,573,양평군,145
625,574,여주시,147


In [32]:
weather_index.drop(['지점명'], axis = 1, inplace = True)

In [33]:
weather_index 

,지점,air_out_idx
0,505,1
1,1524,1
2,1428,1
3,1432,1
4,1256,1
...,...,...
622,547,145
623,326,145
624,573,145
625,574,147


In [34]:
len(weather_index['air_out_idx'].unique())

73

In [ ]:
# weather data 불러오기 - 9분

year_list = np.arange(2012, 2018, 1).tolist()
year_list = list(map(str, year_list))
year_list

weather_data = pd.DataFrame()
for year in year_list: 
    weather_tmp = pd.read_excel('D:\\SNUlab\\0. data\\out of hospital sudden cardiac arrest\\weather_day_year{}.xlsx'.format(year))
    weather_data = pd.concat([weather_data, weather_tmp], axis = 0)

In [ ]:
# 2012는 11, 12월만 뽑기

weather_data = weather_data.loc[weather_data['dt'] >= '2012-11'].reset_index(drop = True)

In [ ]:
# weather_data.to_csv('weather\\weather_1217.csv', encoding = 'euc-kr', index = False)

##### weather_data 다시 불러오기 (12.11 ~ 17.12)

In [36]:
weather_data = pd.read_csv('weather\\weather_1217.csv', encoding = 'euc-kr')

In [37]:
len(weather_data['지점'].unique())

611

In [38]:
weather_data['지점명'].unique()

array(['안면도(감)', '속초', '철원', '독도', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉',
       '강릉', '동해', '서울', '인천', '원주', '울릉도', '관악(레)', '수원', '영월', '충주',
       '서산', '울진', '청주', '대전', '추풍령', '안동', '상주', '포항', '군산', '대구', '전주',
       '울산', '창원', '광주', '부산', '부산(레)', '통영', '목포', '여수', '흑산도', '완도',
       '고창', '순천', '진도(레)', '진도(첨찰산)', '제주', '고산', '성산', '서귀포', '진주',
       '강화', '양평', '이천', '인제', '홍천', '태백', '정선군', '제천', '보은', '북격렬비도',
       '천안', '보령', '부여', '금산', '부안', '임실', '정읍', '남원', '장수', '고창군', '영광군',
       '김해시', '순창군', '북창원', '양산시', '보성군', '강진군', '장흥', '해남', '고흥', '의령군',
       '함양군', '광양시', '봉화', '영주', '문경', '청송군', '영덕', '의성', '구미', '영천',
       '경주시', '거창', '합천', '밀양', '산청', '거제', '남해', '말도', '임자도', '장산도',
       '가거도', '신지도', '여서도', '소리도', '평도', '궁촌', '가야산', '주왕산', '양지암', '덕유봉',
       '성삼재', '무등산', '모악산', '용평', '천부', '향로봉', '원통', '상서', '마현', '송계',
       '백운', '용문산', '우암산', '중문', '산천단', '대흘', '강남', '서초', '강동', '송파',
       '강서', '양천', '도봉', '노원', '동대문', '중랑', '기상청', '마포', '서대문',

In [39]:
# weather_data + weather_index
# pd.set_option("display.max_rows", 253)
weather_data_idx = pd.merge(weather_data, weather_index, on='지점', how = 'inner')

In [40]:
len(weather_data_idx['air_out_idx'].unique())   # 59 + 양평, 여주

61

##### weather_data index 맞추기        
- 없는 지역은 각각 dataframe 만들어서 air_out_idx 만 지역별 idx로 바꿔주고, axis = 0으로 concat
- ex. air_out_idx == 200인 df 6개 만들어서 각각 6개 지역별 idx를 air_out_idx에 넣어줌

54,69,83,148,93,106 -> 200 데이터로 채우기

In [41]:
# df 만들기

weather_54 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()
weather_69 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()
weather_83 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()
weather_148 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()
weather_93 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()
weather_106 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 200].copy()

In [42]:
# 지역별 idx 채우기

weather_54['air_out_idx'] = 54
weather_69['air_out_idx'] = 69
weather_83['air_out_idx'] = 83
weather_148['air_out_idx'] = 148
weather_93['air_out_idx'] = 93
weather_106['air_out_idx'] = 106

127,128,181 -> 125 데이터로 채우기

In [43]:
# df 만들기

weather_127 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 125].copy()
weather_128 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 125].copy()
weather_181 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 125].copy()

In [44]:
# 지역별 idx 채우기

weather_127['air_out_idx'] = 127
weather_128['air_out_idx'] = 128
weather_181['air_out_idx'] = 181

138,139 -> 27 데이터로 채우기

In [45]:
# df 만들기

weather_138 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 27].copy()
weather_139 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 27].copy()

In [46]:
# 지역별 idx 채우기

weather_138['air_out_idx'] = 138
weather_139['air_out_idx'] = 139

15,20,21,47 -> 231 데이터로 채우기
* 01.10 - 21 추가

In [47]:
# df 만들기

weather_15 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 231].copy()
weather_20 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 231].copy()
weather_21 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 231].copy()
weather_47 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 231].copy()

In [48]:
# 지역별 idx 채우기

weather_15['air_out_idx'] = 15
weather_20['air_out_idx'] = 20
weather_21['air_out_idx'] = 21
weather_47['air_out_idx'] = 47

28,92 -> 131 데이터로 채우기

In [49]:
# df 만들기

weather_28 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 131].copy()
weather_92 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 131].copy()

In [50]:
# 지역별 idx 채우기

weather_28['air_out_idx'] = 28
weather_92['air_out_idx'] = 92

40 -> 136 데이터로 채우기

In [51]:
# df 만들기

weather_40 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 136].copy()

In [52]:
# 지역별 idx 채우기

weather_40['air_out_idx'] = 40

169,170 -> 126 데이터로 채우기

In [53]:
# df 만들기

weather_169 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 126].copy()
weather_170 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 126].copy()

In [54]:
# 지역별 idx 채우기

weather_169['air_out_idx'] = 169
weather_170['air_out_idx'] = 170

114, 116 -> 115 데이터로 채우기
* 01.14 116 추가

In [55]:
# df 만들기

weather_114 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 115].copy()
weather_116 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 115].copy()

In [56]:
# 지역별 idx 채우기

weather_114['air_out_idx'] = 114
weather_116['air_out_idx'] = 116

237,122 -> 3 데이터로 채우기

In [57]:
# df 만들기

weather_237 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 3].copy()
weather_122 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 3].copy()

In [58]:
# 지역별 idx 채우기

weather_237['air_out_idx'] = 237
weather_122['air_out_idx'] = 122

113 -> 2 데이터로 채우기

In [59]:
# df 만들기

weather_113 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 2].copy()

In [60]:
# 지역별 idx 채우기

weather_113['air_out_idx'] = 113

168,177 -> 78 데이터로 채우기

In [61]:
# df 만들기

weather_168 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 78].copy()
weather_177 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 78].copy()

In [62]:
# 지역별 idx 채우기

weather_168['air_out_idx'] = 168
weather_177['air_out_idx'] = 177

43 -> 35 데이터로 채우기

In [63]:
# df 만들기

weather_43 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 35].copy()

In [64]:
# 지역별 idx 채우기

weather_43['air_out_idx'] = 43

110,197 -> 196 데이터로 채우기

In [65]:
# df 만들기

weather_110 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 196].copy()
weather_197 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 196].copy()

In [66]:
# 지역별 idx 채우기

weather_110['air_out_idx'] = 110
weather_197['air_out_idx'] = 197

144,6 -> 155 데이터로 채우기

In [67]:
# df 만들기

weather_144 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 155].copy()
weather_6 = weather_data_idx.loc[weather_data_idx['air_out_idx'] == 155].copy()

In [68]:
# 지역별 idx 채우기

weather_144['air_out_idx'] = 144
weather_6['air_out_idx'] = 6

만든 데이터 다 concat

In [69]:
weather_data_idx_final = pd.concat([weather_data_idx, 
weather_54, weather_69, weather_83, weather_148, weather_93, weather_106,
weather_127, weather_128, weather_181,
weather_138, weather_139,
weather_15, weather_20, weather_21, weather_47,
weather_28, weather_92,
weather_40,
weather_169, weather_170,
weather_114, weather_116, 
weather_237, weather_122,
weather_113,
weather_168, weather_177,
weather_43,
weather_110, weather_197, 
weather_144, weather_6
], axis = 0)

##### 최종 air_out_idx 개수 확인

In [70]:
len(weather_data_idx_final['air_out_idx'].unique())   # 116 추가됨, 수도권 75 기준으로 맞춤

75

In [71]:
# 날짜 기준으로 sort

weather_data_idx_final = weather_data_idx_final.sort_values(by=['dt'])
weather_data_idx_final.reset_index(drop = True, inplace = True)

In [72]:
# datetime 형식으로 바꾸기

weather_data_idx_final['dt'] = pd.to_datetime(weather_data_idx_final['dt'])

##### weather_data_idx_final 이상치 처리

In [73]:
weather_data_idx_final.describe()

,지점,temp_count,temp_mean,temp_median,temp_min,temp_max,temp_sum,rain_count,rain_mean,rain_median,rain_min,rain_max,rain_sum,humi_count,humi_mean,humi_median,humi_min,humi_max,humi_sum,air_out_idx
count,307918.000000,307918.000000,307825.000000,307825.00000,307825.000000,307825.000000,307918.000000,307918.000000,279617.000000,279617.000000,279617.000000,279617.000000,307918.000000,307918.000000,213954.000000,213954.000000,213954.000000,213954.000000,307918.000000,307918.000000
mean,459.521399,23.898470,12.021627,11.83664,7.781469,16.807937,287.347453,20.720120,0.159634,0.048030,0.007434,1.148002,2.683875,16.471109,68.997327,70.290370,47.758401,86.464627,1144.846466,110.415198
std,164.951730,1.149769,10.759003,10.62724,11.286823,10.910679,253.251813,7.891278,0.602338,0.337818,0.151798,4.044085,10.394899,11.062099,17.781187,19.652753,21.473598,16.197639,835.371755,68.556464
min,98.000000,0.000000,-1099.900000,-1099.90000,-1099.900000,-1099.900000,-1099.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-999.900000,-999.900000,-3177.600000,-999.900000,-2397.600000,1.000000
25%,411.000000,24.000000,2.700000,2.65000,-1.500000,7.200000,63.700000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.600000,57.712500,33.600000,81.300000,0.000000,47.000000
50%,506.000000,24.000000,13.100000,12.85000,8.300000,18.500000,312.900000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,70.300000,71.500000,46.300000,91.600000,1439.000000,115.000000
75%,548.000000,24.000000,21.600000,21.30000,17.600000,26.500000,517.400000,24.000000,0.000000,0.000000,0.000000,0.500000,0.000000,24.000000,81.600000,85.500000,60.600000,96.900000,1825.200000,155.000000
max,889.000000,24.000000,34.400000,34.40000,34.400000,38.300000,783.300000,24.000000,26.800000,21.000000,21.000000,94.000000,322.000000,24.000000,100.000000,100.000000,100.000000,100.000000,2400.000000,247.000000


In [74]:
# humidity < 0 삭제

weather_data_idx_final = weather_data_idx_final.loc[weather_data_idx_final['humi_mean'] >= 0].reset_index(drop = True)
weather_data_idx_final = weather_data_idx_final.loc[weather_data_idx_final['humi_min'] >= 0].reset_index(drop = True)

In [75]:
weather_data_idx_final.describe()

,지점,temp_count,temp_mean,temp_median,temp_min,temp_max,temp_sum,rain_count,rain_mean,rain_median,rain_min,rain_max,rain_sum,humi_count,humi_mean,humi_median,humi_min,humi_max,humi_sum,air_out_idx
count,213815.000000,213815.000000,213746.000000,213746.000000,213746.000000,213746.000000,213815.000000,213815.000000,186647.000000,186647.000000,186647.000000,186647.000000,213815.000000,213815.000000,213815.00000,213815.000000,213815.000000,213815.000000,213815.00000,213815.000000
mean,439.142301,23.907673,12.065465,11.886788,7.912601,16.756313,288.427897,19.470697,0.182743,0.064139,0.010840,1.202310,2.683273,23.713570,69.10811,70.400535,47.880956,86.584666,1649.32377,111.395693
std,188.581899,1.097206,10.467327,10.334112,10.802518,10.637017,250.850345,8.985458,0.661825,0.394526,0.176681,4.114035,10.356150,2.114853,17.12593,19.058401,19.587502,15.347835,413.66428,66.280240
min,98.000000,0.000000,-20.600000,-22.300000,-27.900000,-16.100000,-494.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,1.000000
25%,402.000000,24.000000,2.800000,2.800000,-1.300000,7.200000,66.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,58.70000,57.750000,33.700000,81.300000,1400.10000,54.000000
50%,503.000000,24.000000,13.200000,12.900000,8.500000,18.400000,314.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,70.30000,71.500000,46.300000,91.600000,1681.40000,115.000000
75%,545.000000,24.000000,21.500000,21.200000,17.600000,26.300000,515.950000,24.000000,0.000000,0.000000,0.000000,0.500000,0.000000,24.000000,81.60000,85.500000,60.600000,96.900000,1949.70000,150.000000
max,889.000000,24.000000,34.400000,34.400000,34.400000,38.300000,783.300000,24.000000,21.000000,21.000000,21.000000,91.000000,286.500000,24.000000,100.00000,100.000000,100.000000,100.000000,2400.00000,247.000000


In [66]:
weather_data_idx_final.to_csv('weather\\weather_data_idx_final.csv', encoding = 'euc-kr', index = False)

##### 여기서 3-2_make weather data_zio.r 로 넘어갈 것

In [ ]:
## 전체 다 긁어서 돌리고 파일 다시 불러오기

##### R에서 만든 weather data 가져오기

In [76]:
weather_data_idx_final = pd.read_csv('weather\\weather_1217_afterR.csv', encoding = 'euc-kr')

In [77]:
len(weather_data_idx_final['air_out_idx'].unique())

75

In [78]:
weather_data_idx_final['dt'] = pd.to_datetime(weather_data_idx_final['dt'])

##### weather data 에 72개 idx 만 남기기

In [ ]:
weather_data_idx_final = pd.merge(idx[['OBJECTID']], weather_data_idx_final, left_on = 'OBJECTID', right_on = 'air_out_idx', how = 'left')

In [82]:
len(weather_data_idx_final['air_out_idx'].unique())

72

#### outcome data

In [3]:
# 1분 소요
asthma = pd.read_csv('outcome\\day_asthma_1317_cnt_3_total1.csv', encoding = 'euc-kr')
rhinitis = pd.read_csv('outcome\\day_rhinitis_1317_cnt_3_total1.csv', encoding = 'euc-kr')
atopic = pd.read_csv('outcome\\day_atopic_1317_cnt_3_total1.csv', encoding = 'euc-kr')

In [84]:
# 수도권만 뽑기 - 1분 소요
asthma_sudogwon = asthma.loc[(asthma['시군구'].str.startswith("서울")) | (asthma['시군구'].str.startswith("경기도") | asthma['시군구'].str.startswith('인천광역시'))].reset_index(drop = True)
rhinitis_sudogwon = rhinitis.loc[(rhinitis['시군구'].str.startswith("서울")) | (rhinitis['시군구'].str.startswith("경기도") | rhinitis['시군구'].str.startswith('인천광역시'))].reset_index(drop = True)
atopic_sudogwon = atopic.loc[(atopic['시군구'].str.startswith("서울")) | (atopic['시군구'].str.startswith("경기도") | atopic['시군구'].str.startswith('인천광역시'))].reset_index(drop = True)

수도권 인덱스 맞추기

In [85]:
# 79개
len(asthma_sudogwon['시군구'].unique())

79

In [86]:
# 측정소명 만들기

asthma_sudogwon['측정소명'] = asthma_sudogwon['시군구'].str.replace("서울특별시 ", "")
asthma_sudogwon['측정소명'] = asthma_sudogwon['측정소명'].str.replace("경기도 ", "")
asthma_sudogwon['측정소명'] = asthma_sudogwon['측정소명'].str.replace("인천광역시 ", "")

rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['시군구'].str.replace("서울특별시 ", "")
rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['측정소명'].str.replace("경기도 ", "")
rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['측정소명'].str.replace("인천광역시 ", "")

atopic_sudogwon['측정소명'] = atopic_sudogwon['시군구'].str.replace("서울특별시 ", "")
atopic_sudogwon['측정소명'] = atopic_sudogwon['측정소명'].str.replace("경기도 ", "")
atopic_sudogwon['측정소명'] = atopic_sudogwon['측정소명'].str.replace("인천광역시 ", "")

In [87]:
# idx 맞추기

# 1. 부천시 소사구, 부천시 오정구, 부천시 원미구 -> 부천시 하나로 통합
asthma_sudogwon['측정소명'] = asthma_sudogwon['측정소명'].str.replace(" 소사구", "")
asthma_sudogwon['측정소명'] = asthma_sudogwon['측정소명'].str.replace(" 오정구", "")
asthma_sudogwon['측정소명'] = asthma_sudogwon['측정소명'].str.replace(" 원미구", "")

rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['측정소명'].str.replace(" 소사구", "")
rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['측정소명'].str.replace(" 오정구", "")
rhinitis_sudogwon['측정소명'] = rhinitis_sudogwon['측정소명'].str.replace(" 원미구", "")

atopic_sudogwon['측정소명'] = atopic_sudogwon['측정소명'].str.replace(" 소사구", "")
atopic_sudogwon['측정소명'] = atopic_sudogwon['측정소명'].str.replace(" 오정구", "")
atopic_sudogwon['측정소명'] = atopic_sudogwon['측정소명'].str.replace(" 원미구", "")

In [88]:
# 2. 인천 강화군, 옹진군 삭제

asthma_sudogwon = asthma_sudogwon.loc[(asthma_sudogwon['측정소명'] != '강화군') & (asthma_sudogwon['측정소명'] != '옹진군')].reset_index(drop = True)
rhinitis_sudogwon = rhinitis_sudogwon.loc[(rhinitis_sudogwon['측정소명'] != '강화군') & (rhinitis_sudogwon['측정소명'] != '옹진군')].reset_index(drop = True)
atopic_sudogwon = atopic_sudogwon.loc[(atopic_sudogwon['측정소명'] != '강화군') & (atopic_sudogwon['측정소명'] != '옹진군')].reset_index(drop = True)

In [89]:
# 3. 인천 남구 -> 미추홀구로 이름 변경

asthma_sudogwon.loc[asthma_sudogwon['측정소명'] == '남구', '측정소명'] = '미추홀구'
rhinitis_sudogwon.loc[rhinitis_sudogwon['측정소명'] == '남구', '측정소명'] = '미추홀구'
atopic_sudogwon.loc[atopic_sudogwon['측정소명'] == '남구', '측정소명'] = '미추홀구'

In [90]:
# 4. 인천광역시 중구, 서울특별시 중구 구분

asthma_sudogwon.loc[asthma_sudogwon['시군구'] == '인천광역시 중구', '측정소명'] = '인천 중구'
asthma_sudogwon.loc[asthma_sudogwon['시군구'] == '서울특별시 중구', '측정소명'] = '서울 중구'

rhinitis_sudogwon.loc[rhinitis_sudogwon['시군구'] == '인천광역시 중구', '측정소명'] = '인천 중구'
rhinitis_sudogwon.loc[rhinitis_sudogwon['시군구'] == '서울특별시 중구', '측정소명'] = '서울 중구'

atopic_sudogwon.loc[atopic_sudogwon['시군구'] == '인천광역시 중구', '측정소명'] = '인천 중구'
atopic_sudogwon.loc[atopic_sudogwon['시군구'] == '서울특별시 중구', '측정소명'] = '서울 중구'

In [91]:
len(asthma_sudogwon['측정소명'].unique()) # 75개 확인

75

In [92]:
# qgis 지도 idx 72개 + outcome data 붙이기
# qgis_idx = pd.read_csv('spatial\\qgis_지도_idx_75.csv', encoding = 'euc-kr')
idx.rename(columns = {'OBJECTID' : 'air_out_idx'}, inplace = True)

# 중구 2개 이름 변경
idx.loc[idx['air_out_idx'] == 200, 'SIGUNGU_NM'] = '인천 중구'
idx.loc[idx['air_out_idx'] == 197, 'SIGUNGU_NM'] = '서울 중구'

asthma_sudogwon = pd.merge(asthma_sudogwon, idx[['SIGUNGU_NM', 'air_out_idx']], left_on='측정소명', right_on='SIGUNGU_NM', how='left')
rhinitis_sudogwon = pd.merge(rhinitis_sudogwon, idx[['SIGUNGU_NM', 'air_out_idx']], left_on='측정소명', right_on='SIGUNGU_NM', how='left')
atopic_sudogwon = pd.merge(atopic_sudogwon, idx[['SIGUNGU_NM', 'air_out_idx']], left_on='측정소명', right_on='SIGUNGU_NM', how='left')

In [93]:
print("asthma unique idx: ", len(asthma_sudogwon['air_out_idx'].unique()))
print("rhinitis unique idx: ", len(rhinitis_sudogwon['air_out_idx'].unique()))
print("atopic unique idx: ", len(atopic_sudogwon['air_out_idx'].unique()))

asthma unique idx:  73
rhinitis unique idx:  73
atopic unique idx:  73


In [96]:
len(asthma_sudogwon)

4780468

In [98]:
# 안붙은 3개 지우기
# '경기도 고양시 일산서구', '경기도 양평군', '경기도 여주시'

asthma_sudogwon = asthma_sudogwon.loc[asthma_sudogwon['air_out_idx'].isna() == False].reset_index(drop = True)
rhinitis_sudogwon = rhinitis_sudogwon.loc[rhinitis_sudogwon['air_out_idx'].isna() == False].reset_index(drop = True)
atopic_sudogwon = atopic_sudogwon.loc[atopic_sudogwon['air_out_idx'].isna() == False].reset_index(drop = True)

In [247]:
print('final 72개 check')
print("asthma unique idx: ", len(asthma_sudogwon['air_out_idx'].unique()))
print("rhinitis unique idx: ", len(rhinitis_sudogwon['air_out_idx'].unique()))
print("atopic unique idx: ", len(atopic_sudogwon['air_out_idx'].unique()))

final 72개 check
asthma unique idx:  72
rhinitis unique idx:  72
atopic unique idx:  72


In [101]:
# date 형식 바꾸기
asthma_sudogwon['date'] = asthma_sudogwon['DT'].astype(str)
rhinitis_sudogwon['date'] = rhinitis_sudogwon['DT'].astype(str)
atopic_sudogwon['date'] = atopic_sudogwon['DT'].astype(str)

asthma_sudogwon['date'] = asthma_sudogwon['date'].str[0:4] + '-' + asthma_sudogwon['date'].str[4:6] + '-' + asthma_sudogwon['date'].str[6:8]
rhinitis_sudogwon['date'] = rhinitis_sudogwon['date'].str[0:4] + '-' + rhinitis_sudogwon['date'].str[4:6] + '-' + rhinitis_sudogwon['date'].str[6:8]
atopic_sudogwon['date'] = atopic_sudogwon['date'].str[0:4] + '-' + atopic_sudogwon['date'].str[4:6] + '-' + atopic_sudogwon['date'].str[6:8]

asthma_sudogwon['date'] = pd.to_datetime(asthma_sudogwon['date'])
rhinitis_sudogwon['date'] = pd.to_datetime(rhinitis_sudogwon['date'])
atopic_sudogwon['date'] = pd.to_datetime(atopic_sudogwon['date'])

In [108]:
asthma_sudogwon

,DT,DAY,ADDR_NUM,AGE,SEX_TYPE,시군구,ASTHMA_out_total,ASTHMA_in_total,ASTHMA_in_2_total,ASTHMA_em_total,측정소명,SIGUNGU_NM,air_out_idx,date
0,20130101,2,1806,0,1,경기도 가평군,0,0,0,0,가평군,가평군,1.0,2013-01-01
1,20130101,2,1812,0,2,경기도 가평군,2,0,0,0,가평군,가평군,1.0,2013-01-01
2,20130101,2,1807,5,1,경기도 가평군,0,0,0,0,가평군,가평군,1.0,2013-01-01
3,20130101,2,1806,5,2,경기도 가평군,2,0,0,0,가평군,가평군,1.0,2013-01-01
4,20130101,2,1812,10,1,경기도 가평군,0,1,1,0,가평군,가평군,1.0,2013-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594211,20171231,7,800,70,2,인천광역시 중구,0,0,6,0,인천 중구,인천 중구,200.0,2017-12-31
4594212,20171231,7,801,75,1,인천광역시 중구,0,0,3,0,인천 중구,인천 중구,200.0,2017-12-31
4594213,20171231,7,800,75,2,인천광역시 중구,0,0,4,0,인천 중구,인천 중구,200.0,2017-12-31
4594214,20171231,7,801,80,1,인천광역시 중구,0,1,4,1,인천 중구,인천 중구,200.0,2017-12-31


#### export outcome, air, weather

In [248]:
# 4분 이상 소요
# final datasets
# 1. air_data_idx
# 2. weather_data_idx_final
# 3. asthma_seoul, rhinitis_seoul, atopic_seoul - raw, 혹시 모를 subgroup을 위해 save

air_data_idx.to_csv('air\\air_data_idx.csv', encoding = 'euc-kr', index = False)
weather_data_idx_final.to_csv('weather\\weather_data_idx_final.csv', encoding = 'euc-kr', index = False)
asthma_sudogwon.to_csv('outcome\\asthma_sudogwon.csv', encoding = 'euc-kr', index = False)
rhinitis_sudogwon.to_csv('outcome\\rhinitis_sudogwon.csv', encoding = 'euc-kr', index = False)
atopic_sudogwon.to_csv('outcome\\atopic_sudogwon.csv', encoding = 'euc-kr', index = False)

#### import again

In [3]:
# 30 sec
# final datasets
# 1. air_data_idx
# 2. weather_data_idx_final
# 3. asthma_seoul, rhinitis_seoul, atopic_seoul - raw, 혹시 모를 subgroup을 위해 save

air_data_idx = pd.read_csv('air\\air_data_idx.csv', encoding = 'euc-kr')
weather_data_idx_final = pd.read_csv('weather\\weather_data_idx_final.csv', encoding = 'euc-kr')
asthma_sudogwon = pd.read_csv('outcome\\asthma_sudogwon.csv', encoding = 'euc-kr')
rhinitis_sudogwon = pd.read_csv('outcome\\rhinitis_sudogwon.csv', encoding = 'euc-kr')
atopic_sudogwon = pd.read_csv('outcome\\atopic_sudogwon.csv', encoding = 'euc-kr')

In [12]:
# 0922 - age group check

asthma_sudogwon = pd.read_csv('outcome\\asthma_sudogwon.csv', encoding = 'euc-kr')
rhinitis_sudogwon = pd.read_csv('outcome\\rhinitis_sudogwon.csv', encoding = 'euc-kr')
atopic_sudogwon = pd.read_csv('outcome\\atopic_sudogwon.csv', encoding = 'euc-kr')

In [13]:
asthma_sudogwon_1517 = asthma_sudogwon.loc[asthma_sudogwon['DT'] >= 20150101].reset_index(drop = True)
rhinitis_sudogwon_1517 = rhinitis_sudogwon.loc[rhinitis_sudogwon['DT'] >= 20150101].reset_index(drop = True)
atopic_sudogwon_1517 = atopic_sudogwon.loc[atopic_sudogwon['DT'] >= 20150101].reset_index(drop = True)

In [ ]:
tmp = atopic_sudogwon_1517.groupby(['AGE']).sum().reset_index()
tmp[['AGE', 'ATOPIC_out_total', 'ATOPIC_in_total', 'ATOPIC_em_total']]

In [ ]:
tmp = asthma_sudogwon_1517.groupby(['AGE']).sum().reset_index()
tmp[['AGE', 'ASTHMA_out_total', 'ASTHMA_in_total', 'ASTHMA_em_total']]

In [ ]:
tmp = rhinitis_sudogwon_1517.groupby(['AGE']).sum().reset_index()
tmp[['AGE', 'RHINITIS_out_total', 'RHINITIS_in_total', 'RHINITIS_em_total']]

### raw outcome dataset을 dt, air_out_idx 별 aggregate + air + weather

In [249]:
# raw outcome dataset을 dt, air_out_idx 별 aggregate + air + weather
from dbfread import DBF

# create sudogwon x date idx
date_idx = pd.DataFrame(air_data_idx['dt'].drop_duplicates()).reset_index(drop = True) # 2012.11 ~ 2017.12
# qgis_idx = pd.read_csv('spatial\\qgis_지도_idx_75.csv', encoding = 'euc-kr')
# qgis_idx.rename(columns = {'OBJECTID' : 'air_out_idx'}, inplace = True)
# sudogwon_idx = qgis_idx[['air_out_idx']]
idx = DBF('spatial\\Z_SOP_BND_SIGUNGU_sudogwon.dbf', encoding = 'euc-kr')
idx = pd.DataFrame(idx)
idx['OBJECTID'] = idx['OBJECTID'].astype(int)
idx.rename(columns = {'OBJECTID' : 'air_out_idx'}, inplace = True)
sudogwon_idx = idx[['air_out_idx']]
date_sudogwon_idx = date_idx.merge(sudogwon_idx, how='cross')


# asthma
asthma_sudogwon_agg = asthma_sudogwon[['date', 'air_out_idx', 'ASTHMA_out_total', 'ASTHMA_in_total', 'ASTHMA_in_2_total', 'ASTHMA_em_total']].groupby(['date', 'air_out_idx']).sum()
asthma_sudogwon_agg = pd.DataFrame(asthma_sudogwon_agg).reset_index()
asthma_sudogwon_agg = pd.merge(date_sudogwon_idx, asthma_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
asthma_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
asthma_sudogwon_agg = pd.merge(asthma_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
asthma_sudogwon_agg = pd.merge(asthma_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

# rhinitis
rhinitis_sudogwon_agg = rhinitis_sudogwon[['date', 'air_out_idx', 'RHINITIS_out_total', 'RHINITIS_in_total', 'RHINITIS_in_2_total', 'RHINITIS_em_total']].groupby(['date', 'air_out_idx']).sum()
rhinitis_sudogwon_agg = pd.DataFrame(rhinitis_sudogwon_agg).reset_index()
rhinitis_sudogwon_agg = pd.merge(date_sudogwon_idx, rhinitis_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
rhinitis_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
rhinitis_sudogwon_agg = pd.merge(rhinitis_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
rhinitis_sudogwon_agg = pd.merge(rhinitis_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

# atopic
atopic_sudogwon_agg = atopic_sudogwon[['date', 'air_out_idx', 'ATOPIC_out_total', 'ATOPIC_in_total', 'ATOPIC_in_2_total', 'ATOPIC_em_total']].groupby(['date', 'air_out_idx']).sum()
atopic_sudogwon_agg = pd.DataFrame(atopic_sudogwon_agg).reset_index()
atopic_sudogwon_agg = pd.merge(date_sudogwon_idx, atopic_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
atopic_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
atopic_sudogwon_agg = pd.merge(atopic_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
atopic_sudogwon_agg = pd.merge(atopic_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

In [7]:
asthma_sudogwon['AGE']

0           0
1           0
2           5
3           5
4          10
           ..
4594211    70
4594212    75
4594213    75
4594214    80
4594215    80
Name: AGE, Length: 4594216, dtype: int64

In [4]:
# for descriptive
# raw outcome dataset을 dt, air_out_idx, age group, sex 별 aggregate + air + weather
from dbfread import DBF

# create sudogwon x date idx
date_idx = pd.DataFrame(air_data_idx['dt'].drop_duplicates()).reset_index(drop = True) # 2012.11 ~ 2017.12
# qgis_idx = pd.read_csv('spatial\\qgis_지도_idx_75.csv', encoding = 'euc-kr')
# qgis_idx.rename(columns = {'OBJECTID' : 'air_out_idx'}, inplace = True)
# sudogwon_idx = qgis_idx[['air_out_idx']]
idx = DBF('spatial\\Z_SOP_BND_SIGUNGU_sudogwon.dbf', encoding = 'euc-kr')
idx = pd.DataFrame(idx)
idx['OBJECTID'] = idx['OBJECTID'].astype(int)
idx.rename(columns = {'OBJECTID' : 'air_out_idx'}, inplace = True)
sudogwon_idx = idx[['air_out_idx']]
date_sudogwon_idx = date_idx.merge(sudogwon_idx, how='cross')


# asthma
asthma_sudogwon_agg = asthma_sudogwon[['date', 'air_out_idx', 'AGE', 'SEX_TYPE', 'ASTHMA_out_total', 'ASTHMA_in_total', 'ASTHMA_in_2_total', 'ASTHMA_em_total']].groupby(['date', 'air_out_idx', 'AGE', 'SEX_TYPE']).sum()
asthma_sudogwon_agg = pd.DataFrame(asthma_sudogwon_agg).reset_index()
asthma_sudogwon_agg = pd.merge(date_sudogwon_idx, asthma_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
asthma_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
asthma_sudogwon_agg = pd.merge(asthma_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
asthma_sudogwon_agg = pd.merge(asthma_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

# rhinitis
rhinitis_sudogwon_agg = rhinitis_sudogwon[['date', 'air_out_idx', 'AGE', 'SEX_TYPE', 'RHINITIS_out_total', 'RHINITIS_in_total', 'RHINITIS_in_2_total', 'RHINITIS_em_total']].groupby(['date', 'air_out_idx', 'AGE', 'SEX_TYPE']).sum()
rhinitis_sudogwon_agg = pd.DataFrame(rhinitis_sudogwon_agg).reset_index()
rhinitis_sudogwon_agg = pd.merge(date_sudogwon_idx, rhinitis_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
rhinitis_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
rhinitis_sudogwon_agg = pd.merge(rhinitis_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
rhinitis_sudogwon_agg = pd.merge(rhinitis_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

# atopic
atopic_sudogwon_agg = atopic_sudogwon[['date', 'air_out_idx', 'AGE', 'SEX_TYPE', 'ATOPIC_out_total', 'ATOPIC_in_total', 'ATOPIC_in_2_total', 'ATOPIC_em_total']].groupby(['date', 'air_out_idx', 'AGE', 'SEX_TYPE']).sum()
atopic_sudogwon_agg = pd.DataFrame(atopic_sudogwon_agg).reset_index()
atopic_sudogwon_agg = pd.merge(date_sudogwon_idx, atopic_sudogwon_agg, left_on = ['dt', 'air_out_idx'], right_on = ['date', 'air_out_idx'], how = 'left')
atopic_sudogwon_agg.drop(['date'], axis = 1, inplace = True)
atopic_sudogwon_agg = pd.merge(atopic_sudogwon_agg, air_data_idx, on = ['dt', 'air_out_idx'], how = 'left')
atopic_sudogwon_agg = pd.merge(atopic_sudogwon_agg, weather_data_idx_final, on = ['dt', 'air_out_idx'], how = 'left')

In [8]:
asthma_sudogwon_agg_1517 = asthma_sudogwon_agg.loc[asthma_sudogwon_agg['dt'] >= '2014-12-01'].reset_index(drop = True)
rhinitis_sudogwon_agg_1517 = rhinitis_sudogwon_agg.loc[rhinitis_sudogwon_agg['dt'] >= '2014-12-01'].reset_index(drop = True)
atopic_sudogwon_agg_1517 = atopic_sudogwon_agg.loc[atopic_sudogwon_agg['dt'] >= '2014-12-01'].reset_index(drop = True)

In [9]:
asthma_sudogwon_agg_1517

,dt,air_out_idx,AGE,SEX_TYPE,ASTHMA_out_total,ASTHMA_in_total,ASTHMA_in_2_total,ASTHMA_em_total,측정소코드,SO2_count,...,humi_mean,humi_median,humi_min,humi_max,humi_sum,temp_tc,temp_mean_total,temp_min_total,temp_tc_total,humi_mean_total
0,2014-12-01,1,0.0,1.0,10.0,0.0,0.0,0.0,131611.0,24.0,...,57.1,52.3,38.1,80.4,1371.2,16.3,0.40,-7.00,16.00,56.85
1,2014-12-01,1,0.0,2.0,8.0,2.0,4.0,0.0,131611.0,24.0,...,57.1,52.3,38.1,80.4,1371.2,16.3,0.40,-7.00,16.00,56.85
2,2014-12-01,1,5.0,1.0,8.0,0.0,0.0,0.0,131611.0,24.0,...,57.1,52.3,38.1,80.4,1371.2,16.3,0.40,-7.00,16.00,56.85
3,2014-12-01,1,5.0,2.0,5.0,0.0,0.0,0.0,131611.0,24.0,...,57.1,52.3,38.1,80.4,1371.2,16.3,0.40,-7.00,16.00,56.85
4,2014-12-01,1,10.0,1.0,0.0,0.0,0.0,0.0,131611.0,24.0,...,57.1,52.3,38.1,80.4,1371.2,16.3,0.40,-7.00,16.00,56.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758891,2017-12-31,247,70.0,2.0,2.0,1.0,9.0,1.0,131552.0,24.0,...,72.8,69.9,41.9,99.9,1746.5,5.2,0.65,-1.65,5.25,67.60
2758892,2017-12-31,247,75.0,1.0,1.0,0.0,5.0,0.0,131552.0,24.0,...,72.8,69.9,41.9,99.9,1746.5,5.2,0.65,-1.65,5.25,67.60
2758893,2017-12-31,247,75.0,2.0,1.0,0.0,11.0,0.0,131552.0,24.0,...,72.8,69.9,41.9,99.9,1746.5,5.2,0.65,-1.65,5.25,67.60
2758894,2017-12-31,247,80.0,1.0,2.0,0.0,13.0,0.0,131552.0,24.0,...,72.8,69.9,41.9,99.9,1746.5,5.2,0.65,-1.65,5.25,67.60


In [9]:
print(len(asthma_sudogwon_agg['air_out_idx'].unique()))
print(len(rhinitis_sudogwon_agg['air_out_idx'].unique()))
print(len(atopic_sudogwon_agg['air_out_idx'].unique()))

72
72
72


In [10]:
print(len(asthma_sudogwon_agg['dt'].unique()))

1887


In [252]:
365 * 5 + 62

1887

In [ ]:
idx_list = asthma_sudogwon_agg['air_out_idx'].unique().tolist()
for ix in idx_list: 
    tmp = asthma_sudogwon_agg.loc[asthma_sudogwon_agg['air_out_idx'] == ix]
    print(ix, '---------------------------')
    for col in tmp.columns: 
        print(col, tmp[col].isna().sum())
    print('-------------------------')

In [253]:
# export - 1분 소요

asthma_sudogwon_agg.to_csv('outcome\\asthma_sudogwon_agg.csv', encoding = 'euc-kr', index = False)
rhinitis_sudogwon_agg.to_csv('outcome\\rhinitis_sudogwon_agg.csv', encoding = 'euc-kr', index = False)
atopic_sudogwon_agg.to_csv('outcome\\atopic_sudogwon_agg.csv', encoding = 'euc-kr', index = False)

In [12]:
# export - raw data + air + weather for descriptive
# 13분 소요

asthma_sudogwon_agg_1517.to_csv('outcome\\asthma_sudogwon_with_air_weather.csv', encoding = 'euc-kr', index = False)
rhinitis_sudogwon_agg_1517.to_csv('outcome\\rhinitis_sudogwon_with_air_weather.csv', encoding = 'euc-kr', index = False)
atopic_sudogwon_agg_1517.to_csv('outcome\\atopic_sudogwon_with_air_weather.csv', encoding = 'euc-kr', index = False)

-------------------- end.                
move to 4_eigen filter_zio.R